In [ ]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
from rdflib import Graph, URIRef, Literal
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
def load_foodon_graph():
    # Load the FoodOn ontology
    g = Graph()
    g.parse("https://raw.githubusercontent.com/FoodOntology/foodon/master/foodon.owl", format="xml")
    return g

In [ ]:
def get_foodon_triples(graph):
    triples = []
    for s, p, o in graph:
        if isinstance(s, URIRef) and isinstance(p, URIRef):
            # Filter out literals with datetime or other unsupported formats
            if isinstance(o, Literal):
                try:
                    if o.datatype and 'dateTime' in o.datatype:
                        continue
                    if isinstance(o.value, (str, int, float)):
                        triples.append((str(s), str(p), str(o)))
                except Exception as e:
                    logger.warning(f"Skipping literal with parsing issue: {o} - {e}")
            else:
                triples.append((str(s), str(p), str(o)))
    return triples

In [ ]:
def map_uris_to_indices(triples):
    entity2id = {}
    relation2id = {}
    entity_idx = 0
    relation_idx = 0

    indexed_triples = []

    for s, p, o in triples:
        if s not in entity2id:
            entity2id[s] = entity_idx
            entity_idx += 1
        if o not in entity2id:
            entity2id[o] = entity_idx
            entity_idx += 1
        if p not in relation2id:
            relation2id[p] = relation_idx
            relation_idx += 1

        indexed_triples.append((entity2id[s], relation2id[p], entity2id[o]))

    return indexed_triples, entity2id, relation2id

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

In [ ]:
class KeplerModel(nn.Module):
    def __init__(self, pretrained_model_name='bert-base-uncased', entity_vocab_size=10000, relation_vocab_size=500):
        super(KeplerModel, self).__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        self.entity_embeddings = nn.Embedding(entity_vocab_size, self.bert.config.hidden_size)
        self.relation_embeddings = nn.Embedding(relation_vocab_size, self.bert.config.hidden_size)
        self.linear = nn.Linear(self.bert.config.hidden_size, self.bert.config.vocab_size)
        self.loss_fn_mlm = nn.CrossEntropyLoss()
        self.loss_fn_kge = nn.MSELoss()  # Simple loss function for KGE; you can replace it with more sophisticated ones

    def forward(self, input_ids, attention_mask, token_type_ids, labels=None, triples=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs[0]
        logits = self.linear(sequence_output)

        if labels is not None:
            logits = logits.view(-1, self.bert.config.vocab_size)
            labels = labels.view(-1)
            loss_mlm = self.loss_fn_mlm(logits, labels)
        else:
            loss_mlm = None

        if triples is not None:
            triples = triples.to(input_ids.device)  # Ensure triples are on the same device as the input tensors
            head_embeddings = self.get_entity_embeddings(triples[:, 0])
            relation_embeddings = self.get_relation_embeddings(triples[:, 1])
            tail_embeddings = self.get_entity_embeddings(triples[:, 2])
            loss_kge = self.loss_fn_kge(head_embeddings + relation_embeddings, tail_embeddings)
        else:
            loss_kge = None

        return loss_mlm, loss_kge

    def get_entity_embeddings(self, entities):
        return self.entity_embeddings(entities)

    def get_relation_embeddings(self, relations):
        return self.relation_embeddings(relations)

    def save_pretrained(self, save_path):
        self.bert.save_pretrained(save_path)
        torch.save(self.linear.state_dict(), save_path + '/linear.pth')

In [ ]:
import json
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer

In [ ]:
class Recipe1MDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=512):
        self.data = self.load_data(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def load_data(self, file_path):
        data = []
        with open(file_path, 'r') as f:
            for line in f:
                item = json.loads(line)
                data.append(item)
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        recipe = self.data[idx]
        title = recipe['title']
        ingredients = ' '.join([ingredient['text'] for ingredient in recipe['ingredients']])
        instructions = ' '.join([instruction['text'] for instruction in recipe['instructions']])
        text = f"{title} [SEP] {ingredients} [SEP] {instructions}"

        tokens = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")
        input_ids = tokens['input_ids'].squeeze()
        attention_mask = tokens['attention_mask'].squeeze()
        token_type_ids = tokens['token_type_ids'].squeeze()

        return input_ids, attention_mask, token_type_ids, input_ids  # Using input_ids as labels for MLM

In [ ]:
import json
import torch
from transformers import BertTokenizer
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
# import torch_xla
# import torch_xla.core.xla_model as xm
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import os
import datetime

In [ ]:
def load_config(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
    return config

In [ ]:
def collate_fn(batch):
    input_ids = torch.stack([item[0] for item in batch])
    attention_masks = torch.stack([item[1] for item in batch])
    token_type_ids = torch.stack([item[2] for item in batch])
    labels = torch.stack([item[3] for item in batch])
    return input_ids, attention_masks, token_type_ids, labels

In [ ]:
def save_metrics(metrics, file_path):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as f:
        json.dump(metrics, f, indent=4)

def calculate_metrics(predictions, labels):
    predictions = predictions.argmax(dim=-1).cpu().numpy().flatten()
    labels = labels.cpu().numpy().flatten()
    precision = precision_score(labels, predictions, average='weighted', zero_division=0)
    recall = recall_score(labels, predictions, average='weighted', zero_division=0)
    f1 = f1_score(labels, predictions, average='weighted', zero_division=0)
    return precision, recall, f1

In [ ]:
def save_checkpoint(model, optimizer, epoch, save_path, scaler=None):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict() if scaler else None
    }
    os.makedirs(save_path, exist_ok=True)
    checkpoint_path = os.path.join(save_path, f'checkpoint_epoch_{epoch}.pth')
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved at {checkpoint_path}")

In [ ]:
# TRAIN FUNCTION INITIAL
def train(config_path):
    print("loading config")
    config = load_config(config_path)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    print("loaded config and tokenizer: \n", config)

    # Load and prepare the FoodOn triples
    print("loading FoodOn graph")
    foodon_graph = load_foodon_graph()
    foodon_triples = get_foodon_triples(foodon_graph)
    indexed_triples, entity2id, relation2id = map_uris_to_indices(foodon_triples)
    foodon_triples_tensor = torch.tensor(indexed_triples, dtype=torch.long)
    print("loaded FoodOn graph and mapped URIs to indices")

    # Update model initialization with entity and relation vocab sizes
    print("initializing model")
    model = KeplerModel(
        pretrained_model_name='bert-base-uncased',
        entity_vocab_size=len(entity2id),
        relation_vocab_size=len(relation2id)
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # xm.xla_device()
    model.to(device)

    foodon_triples_tensor = foodon_triples_tensor.to(device)  # Ensure triples tensor is on the same device as the model

    train_dataset = Recipe1MDataset(file_path=config['train_data_path'], tokenizer=tokenizer)
    valid_dataset = Recipe1MDataset(file_path=config['valid_data_path'], tokenizer=tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, collate_fn=collate_fn)
    valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=False, collate_fn=collate_fn)

    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    scaler = GradScaler()

    metrics = {
        'train_loss': [],
        'val_loss': [],
        'precision': [],
        'recall': [],
        'f1_score': [],
        'step_train_loss': [],
    }

    print("starting training")
    print("device: ", device)
    # print the timestamp
    import datetime
    print(datetime.datetime.now())
    prev_time = datetime.datetime.now()
    # clear torch cache
    torch.cuda.empty_cache()
    step = 0
    for epoch in range(config['max_epochs']):
        model.train()
        total_loss = 0
        for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(train_loader):
            input_ids, attention_mask, token_type_ids, labels = input_ids.to(device), attention_mask.to(device), token_type_ids.to(device), labels.to(device)

            optimizer.zero_grad()
            with autocast():
                loss_mlm, loss_kge = model(input_ids, attention_mask, token_type_ids, labels=labels, triples=foodon_triples_tensor)
            loss = loss_mlm + loss_kge
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
            metrics['step_train_loss'].append(loss.item())

            if step % 100 == 0:
                # print step number and loss with timestamp
                diff_time = datetime.datetime.now()
                print('Elapsed: ', (diff_time - prev_time).total_seconds())
                prev_time = diff_time
                print(f"Step [{step}/{len(train_loader)*config['max_epochs']}], Loss: {loss.item():.4f}")
                save_metrics(metrics, os.path.join(config['save_path'], 'metrics.json'))

            step += 1

        avg_loss = total_loss / len(train_loader)
        metrics['train_loss'].append(avg_loss)
        print(f"Epoch [{epoch+1}/{config['max_epochs']}], Loss: {avg_loss:.4f}")

        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(valid_loader):
                input_ids, attention_mask, token_type_ids, labels = input_ids.to(device), attention_mask.to(device), token_type_ids.to(device), labels.to(device)
                with autocast():
                    loss_mlm, loss_kge = model(input_ids, attention_mask, token_type_ids, labels=labels, triples=foodon_triples_tensor)
                val_loss += (loss_mlm + loss_kge).item()

                # Store predictions and labels for metric calculation
                # logits = model(input_ids, attention_mask, token_type_ids)[0]
                # all_preds.append(logits)
                # all_labels.append(labels)

        avg_val_loss = val_loss / len(valid_loader)
        metrics['val_loss'].append(avg_val_loss)
        print(f"Validation Loss: {avg_val_loss:.4f}")

        # Calculate precision, recall, f1 score
        # all_preds = torch.cat(all_preds, dim=0)
        # all_labels = torch.cat(all_labels, dim=0)
        # precision, recall, f1 = calculate_metrics(all_preds, all_labels)
        # metrics['precision'].append(precision)
        # metrics['recall'].append(recall)
        # metrics['f1_score'].append(f1)
        # print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

        if epoch % config['save_every'] == 0:
            model.save_pretrained(config['save_path'] + f"epoch_{epoch}")
            # Save embeddings and mappings
            np.save(os.path.join(config['save_path'], 'entity_embeddings.npy'), foodon_triples_tensor.cpu().numpy())
            with open(os.path.join(config['save_path'], 'entity2id.json'), 'w') as f:
                json.dump(entity2id, f)

        # Save metrics at the end of each epoch
        save_metrics(metrics, os.path.join(config['save_path'], 'metrics.json'))

        # Save Checkpoint
        save_checkpoint(model, optimizer, epoch, config['save_path'], scaler=scaler)

    print("training complete")

In [ ]:
# clear torch cache
torch.cuda.empty_cache()
config_path = "config_drive.json"
train(config_path)

loading config


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

loaded config and tokenizer: 
 {'model_name': 'kepler-recipe1m', 'train_data_path': '/content/drive/MyDrive/Repos/foodkg/data/train.jsonl', 'valid_data_path': '/content/drive/MyDrive/Repos/foodkg/data/valid.jsonl', 'test_data_path': '/content/drive/MyDrive/Repos/foodkg/data/test.jsonl', 'pretrained_model_path': '', 'batch_size': 64, 'learning_rate': 1e-05, 'max_epochs': 3, 'save_path': '/content/drive/MyDrive/Repos/foodkg/trained_mixed_model_trial_3/', 'save_every': 1}
loading FoodOn graph


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/isodate/isodatetime.py", line 51, in parse_datetime
    datestring, timestring = datetimestring.split('T')
ValueError: not enough values to unpack (expected 2, got 1)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/usr/local/lib/python3.10/dist-packages/isodate/isodatetime.py", line 53, in parse_datetime
    raise ISO8601Error("ISO 8601 time designator 'T' missing. Unable to"
isodate.isoerror.ISO8601Error: ISO 8601 time designator 'T' missing. Unable to parse datetime string '2024-07-04 17:32:19'


loaded FoodOn graph and mapped URIs to indices
initializing model


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

starting training
device:  cuda
2024-08-12 02:23:17.768142
Elapsed:  2.202599
Step [0/38616], Loss: 13.1847
Elapsed:  55.801646
Step [100/38616], Loss: 7.4195
Elapsed:  55.388047
Step [200/38616], Loss: 6.4637
Elapsed:  55.542688
Step [300/38616], Loss: 5.9391
Elapsed:  55.646824
Step [400/38616], Loss: 5.2957
Elapsed:  55.526915
Step [500/38616], Loss: 4.8274
Elapsed:  55.773926
Step [600/38616], Loss: 4.8105
Elapsed:  55.390391
Step [700/38616], Loss: 4.4406
Elapsed:  55.84675
Step [800/38616], Loss: 4.2433
Elapsed:  55.481455
Step [900/38616], Loss: 3.8567
Elapsed:  55.535006
Step [1000/38616], Loss: 3.8128
Elapsed:  55.736672
Step [1100/38616], Loss: 3.6477
Elapsed:  55.54678
Step [1200/38616], Loss: 3.5300
Elapsed:  55.626481
Step [1300/38616], Loss: 3.4967
Elapsed:  55.20404
Step [1400/38616], Loss: 3.4567
Elapsed:  55.528401
Step [1500/38616], Loss: 3.3020
Elapsed:  55.616987
Step [1600/38616], Loss: 3.3466
Elapsed:  55.564126
Step [1700/38616], Loss: 3.2677
Elapsed:  55.565387


In [ ]:
# TRAIN FUNCTION WITH CONTINUE FUNCTIONALITY

def train(config_path, continue_training=False, checkpoint_path=None):
    print("loading config")
    config = load_config(config_path)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    print("loaded config and tokenizer: \n", config)

    # Load and prepare the FoodOn triples
    print("loading FoodOn graph")
    foodon_graph = load_foodon_graph()
    foodon_triples = get_foodon_triples(foodon_graph)
    indexed_triples, entity2id, relation2id = map_uris_to_indices(foodon_triples)
    foodon_triples_tensor = torch.tensor(indexed_triples, dtype=torch.long)
    print("loaded FoodOn graph and mapped URIs to indices")

    if continue_training and checkpoint_path:
        # Load the model, optimizer, and scaler state from the checkpoint
        print("loading checkpoint from ", checkpoint_path)
        checkpoint = torch.load(checkpoint_path)

        entity_vocab_size = checkpoint['model_state_dict']['entity_embeddings.weight'].shape[0]
        relation_vocab_size = checkpoint['model_state_dict']['relation_embeddings.weight'].shape[0]

        model = KeplerModel(
            pretrained_model_name='bert-base-uncased',
            entity_vocab_size=entity_vocab_size,
            relation_vocab_size=relation_vocab_size
        )
        model.load_state_dict(checkpoint['model_state_dict'])

        optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        scaler = GradScaler()
        scaler.load_state_dict(checkpoint['scaler_state_dict'])

        # Ensure model and optimizer are on the correct device
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # Move optimizer states to the correct device
        for state in optimizer.state.values():
            if isinstance(state, torch.Tensor):
                state.data = state.data.to(device)
            elif isinstance(state, dict):
                for k, v in state.items():
                    if isinstance(v, torch.Tensor):
                        state[k] = v.to(device)

        start_epoch = checkpoint['epoch'] + 1
        print(f"Resuming training from epoch {start_epoch}")
    else:
        # Initialize model, optimizer, and scaler from scratch
        print("initializing model")
        model = KeplerModel(
            pretrained_model_name='bert-base-uncased',
            entity_vocab_size=len(entity2id),
            relation_vocab_size=len(relation2id)
        )

        optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
        scaler = GradScaler()

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        start_epoch = 0

    foodon_triples_tensor = foodon_triples_tensor.to(device)  # Ensure triples tensor is on the same device as the model

    train_dataset = Recipe1MDataset(file_path=config['train_data_path'], tokenizer=tokenizer)
    valid_dataset = Recipe1MDataset(file_path=config['valid_data_path'], tokenizer=tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, collate_fn=collate_fn)
    valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=False, collate_fn=collate_fn)

    metrics = {
        'train_loss': [],
        'val_loss': [],
        'precision': [],
        'recall': [],
        'f1_score': [],
        'step_train_loss': [],
    }

    print("starting training")
    print("device: ", device)
    print(datetime.datetime.now())
    prev_time = datetime.datetime.now()
    torch.cuda.empty_cache()

    step = 0
    for epoch in range(start_epoch, start_epoch + config['max_epochs']):
        model.train()
        total_loss = 0
        for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(train_loader):
            input_ids, attention_mask, token_type_ids, labels = input_ids.to(device), attention_mask.to(device), token_type_ids.to(device), labels.to(device)

            optimizer.zero_grad()
            with autocast():
                loss_mlm, loss_kge = model(input_ids, attention_mask, token_type_ids, labels=labels, triples=foodon_triples_tensor)
            loss = loss_mlm + loss_kge
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
            metrics['step_train_loss'].append(loss.item())

            if step % 100 == 0:
                diff_time = datetime.datetime.now()
                print('Elapsed: ', (diff_time - prev_time).total_seconds())
                prev_time = diff_time
                print(f"Step [{step}/{len(train_loader)*config['max_epochs']}], Loss: {loss.item():.4f}")
                save_metrics(metrics, os.path.join(config['save_path'], 'metrics.json'))

            step += 1

        avg_loss = total_loss / len(train_loader)
        metrics['train_loss'].append(avg_loss)
        print(f"Epoch [{epoch+1}/{config['max_epochs']}], Loss: {avg_loss:.4f}")

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(valid_loader):
                input_ids, attention_mask, token_type_ids, labels = input_ids.to(device), attention_mask.to(device), token_type_ids.to(device), labels.to(device)
                with autocast():
                    loss_mlm, loss_kge = model(input_ids, attention_mask, token_type_ids, labels=labels, triples=foodon_triples_tensor)
                val_loss += (loss_mlm + loss_kge).item()

        avg_val_loss = val_loss / len(valid_loader)
        metrics['val_loss'].append(avg_val_loss)
        print(f"Validation Loss: {avg_val_loss:.4f}")

        if epoch % config['save_every'] == 0:
            model.save_pretrained(config['save_path'] + f"epoch_{epoch}")
            np.save(os.path.join(config['save_path'], 'entity_embeddings.npy'), foodon_triples_tensor.cpu().numpy())
            with open(os.path.join(config['save_path'], 'entity2id.json'), 'w') as f:
                json.dump(entity2id, f)

        save_metrics(metrics, os.path.join(config['save_path'], 'metrics.json'))
        save_checkpoint(model, optimizer, epoch, config['save_path'], scaler=scaler)

    print("training complete")


In [ ]:
# clear torch cache
torch.cuda.empty_cache()
config_path = "config_drive.json"
train(config_path, continue_training=True, checkpoint_path='/content/drive/MyDrive/Repos/foodkg/trained_mixed_model_trial_3/checkpoint_epoch_2.pth')

loading config


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

loaded config and tokenizer: 
 {'model_name': 'kepler-recipe1m', 'train_data_path': '/content/drive/MyDrive/Repos/foodkg/data/train.jsonl', 'valid_data_path': '/content/drive/MyDrive/Repos/foodkg/data/valid.jsonl', 'test_data_path': '/content/drive/MyDrive/Repos/foodkg/data/test.jsonl', 'pretrained_model_path': '', 'batch_size': 64, 'learning_rate': 1e-05, 'max_epochs': 2, 'save_path': '/content/drive/MyDrive/Repos/foodkg/trained_mixed_model_trial_4/', 'save_every': 1}
loading FoodOn graph


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/isodate/isodatetime.py", line 51, in parse_datetime
    datestring, timestring = datetimestring.split('T')
ValueError: not enough values to unpack (expected 2, got 1)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/usr/local/lib/python3.10/dist-packages/isodate/isodatetime.py", line 53, in parse_datetime
    raise ISO8601Error("ISO 8601 time designator 'T' missing. Unable to"
isodate.isoerror.ISO8601Error: ISO 8601 time designator 'T' missing. Unable to parse datetime string '2024-07-04 17:32:19'


loaded FoodOn graph and mapped URIs to indices
loading checkpoint from  /content/drive/MyDrive/Repos/foodkg/trained_mixed_model_trial_3/checkpoint_epoch_2.pth


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Resuming training from epoch 3
starting training
device:  cuda
2024-08-14 22:45:13.355341
Elapsed:  2.062813
Step [0/25744], Loss: 2.0056
Elapsed:  56.179492
Step [100/25744], Loss: 1.9956
Elapsed:  55.948311
Step [200/25744], Loss: 1.9880
Elapsed:  56.081575
Step [300/25744], Loss: 1.9818
Elapsed:  55.767666
Step [400/25744], Loss: 1.9762
Elapsed:  56.114503
Step [500/25744], Loss: 1.9708
Elapsed:  55.881542
Step [600/25744], Loss: 1.9665
Elapsed:  56.184013
Step [700/25744], Loss: 1.9614
Elapsed:  55.908217
Step [800/25744], Loss: 1.9565
Elapsed:  55.826037
Step [900/25744], Loss: 1.9522
Elapsed:  56.077095
Step [1000/25744], Loss: 1.9475
Elapsed:  55.808886
Step [1100/25744], Loss: 1.9435
Elapsed:  56.019725
Step [1200/25744], Loss: 1.9392
Elapsed:  55.977624
Step [1300/25744], Loss: 1.9351
Elapsed:  56.062054
Step [1400/25744], Loss: 1.9309
Elapsed:  55.648358
Step [1500/25744], Loss: 1.9272
Elapsed:  56.040068
Step [1600/25744], Loss: 1.9228
Elapsed:  55.948439
Step [1700/25744], 